DengAI: Predicting Disease Spread


In [200]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
pd.set_option('display.max_columns',None)

# Igore warning statements
import warnings
warnings.filterwarnings('ignore')



In [201]:
X = pd.read_csv('dengue_features_train.csv')
y = pd.read_csv('dengue_labels_train.csv')
df_test = pd.read_csv('dengue_features_test.csv')
df_test_iq = df_test[df_test['city']=='iq']
df_test_sj = df_test[df_test['city']=='sj']
holdout = df_test.copy()

df = pd.merge(X,y, left_on=['city','weekofyear','year'],right_on=['city','weekofyear','year'])
df_iq = df[df['city']=='iq']
df_sj = df[df['city']=='sj']
holdout_iq = holdout[holdout['city']=='iq']
holdout_sj = holdout[holdout['city']=='sj']


In [202]:
df.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
count,1456.000000,1456.000000,1262.000000,1404.000000,1434.000000,1434.000000,1443.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1443.000000,1446.000000,1446.000000,1413.000000,1413.000000,1436.000000,1442.000000,1434.000000,1456.000000
mean,2001.031593,26.503434,0.142294,0.130553,0.203783,0.202305,45.760388,298.701852,299.225578,295.246356,303.427109,295.719156,40.151819,82.161959,45.760388,16.746427,4.903754,27.185783,8.059328,32.452437,22.102150,39.326360,24.675137
std,5.408314,15.019437,0.140531,0.119999,0.073860,0.083903,43.715537,1.362420,1.261715,1.527810,3.234601,2.565364,43.434399,7.153897,43.715537,1.542494,3.546445,1.292347,2.128568,1.959318,1.574066,47.455314,43.596000
min,1990.000000,1.000000,-0.406250,-0.456100,-0.015533,-0.063457,0.000000,294.635714,294.892857,289.642857,297.800000,286.900000,0.000000,57.787143,0.000000,11.715714,1.357143,21.400000,4.528571,26.700000,14.700000,0.000000,0.000000
25%,1997.000000,13.750000,0.044950,0.049217,0.155087,0.144209,9.800000,297.658929,298.257143,294.118929,301.000000,293.900000,13.055000,77.177143,9.800000,15.557143,2.328571,26.300000,6.514286,31.100000,21.100000,8.700000,5.000000
50%,2002.000000,26.500000,0.128817,0.121429,0.196050,0.189450,38.340000,298.646429,299.289286,295.640714,302.400000,296.200000,27.245000,80.301429,38.340000,17.087143,2.857143,27.414286,7.300000,32.800000,22.200000,23.850000,12.000000
75%,2005.000000,39.250000,0.248483,0.216600,0.248846,0.246982,70.235000,299.833571,300.207143,296.460000,305.500000,297.900000,52.200000,86.357857,70.235000,17.978214,7.625000,28.157143,9.566667,33.900000,23.300000,53.900000,28.000000
max,2010.000000,53.000000,0.508357,0.454429,0.538314,0.546017,390.600000,302.200000,302.928571,298.450000,314.000000,299.900000,570.500000,98.610000,390.600000,20.461429,16.028571,30.800000,15.800000,42.200000,25.600000,543.300000,461.000000


In [203]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
ndvi_ne,194,0.133242
ndvi_nw,52,0.035714
station_diur_temp_rng_c,43,0.029533
station_avg_temp_c,43,0.029533
ndvi_se,22,0.015110
ndvi_sw,22,0.015110
station_precip_mm,22,0.015110
station_max_temp_c,20,0.013736
station_min_temp_c,14,0.009615
reanalysis_sat_precip_amt_mm,13,0.008929


In [204]:
df.info()
df_iq.drop(['week_start_date'],inplace=True,axis=1)
df_sj.drop(['week_start_date'],inplace=True,axis=1)
holdout_sj.drop(['week_start_date'],inplace=True,axis=1)
holdout_iq.drop(['week_start_date'],inplace=True,axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1455
Data columns (total 25 columns):
city                                     1456 non-null object
year                                     1456 non-null int64
weekofyear                               1456 non-null int64
week_start_date                          1456 non-null object
ndvi_ne                                  1262 non-null float64
ndvi_nw                                  1404 non-null float64
ndvi_se                                  1434 non-null float64
ndvi_sw                                  1434 non-null float64
precipitation_amt_mm                     1443 non-null float64
reanalysis_air_temp_k                    1446 non-null float64
reanalysis_avg_temp_k                    1446 non-null float64
reanalysis_dew_point_temp_k              1446 non-null float64
reanalysis_max_air_temp_k                1446 non-null float64
reanalysis_min_air_temp_k                1446 non-null float64
reanalysis_precip

In [205]:
from sklearn.model_selection import train_test_split

X_train_sj,X_test_sj, y_train_sj, y_test_sj = train_test_split(df_sj.iloc[:,0:-1],df_sj['total_cases'], test_size=0.2, random_state=123)
X_train_sj.shape
X_train_iq,X_test_iq, y_train_iq, y_test_iq = train_test_split(df_iq.iloc[:,0:-1],df_iq['total_cases'], test_size=0.2, random_state=123)
X_train_iq.shape

(416, 23)

In [206]:
y_train_iq.shape

(416,)

In [207]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN',strategy='mean', axis=0)

imputer = imputer.fit(X_train_sj.iloc[:,2:])
X_train_sj.iloc[:,2:]= imputer.transform(X_train_sj.iloc[:,2:])
imputer = imputer.fit(X_test_sj.iloc[:,2:])
X_test_sj.iloc[:,2:]= imputer.transform(X_test_sj.iloc[:,2:])

imputer = imputer.fit(X_train_iq.iloc[:,2:])
X_train_iq.iloc[:,2:]= imputer.transform(X_train_iq.iloc[:,2:])
imputer = imputer.fit(X_test_iq.iloc[:,2:])
X_test_iq.iloc[:,2:]= imputer.transform(X_test_iq.iloc[:,2:])


imputer = imputer.fit(holdout_sj.iloc[:,2:])
holdout_sj.iloc[:,2:]= imputer.transform(holdout_sj.iloc[:,2:])
imputer = imputer.fit(holdout_sj.iloc[:,2:])
holdout_iq.iloc[:,2:]= imputer.transform(holdout_iq.iloc[:,2:])

In [208]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
sc.fit(X_train_sj.iloc[:,3:])
sc.transform(X_train_sj.iloc[:,3:])
sc.fit(X_train_iq.iloc[:,3:])
sc.transform(X_train_iq.iloc[:,3:])

sc.fit(X_test_sj.iloc[:,3:])
sc.transform(X_test_sj.iloc[:,3:])
sc.fit(X_train_iq.iloc[:,3:])
sc.transform(X_test_iq.iloc[:,3:])


array([[ 1.00559317,  0.14598614,  0.34923782, ..., -1.25970647,
        -2.13777927,  0.17469206],
       [-1.29648727, -0.65405344, -0.45124472, ..., -0.21285079,
         0.73805212,  1.64041332],
       [-1.07232151, -1.39872971, -1.79155413, ..., -0.13807538,
         0.25874689, -0.08641457],
       ..., 
       [ 1.47089492,  0.21589375,  1.01606082, ...,  0.01147543,
        -0.38032675, -0.94822663],
       [-0.9048616 , -0.12978563, -0.9879239 , ..., -0.96060485,
         0.57828371,  0.53351344],
       [-0.78265045, -0.62085784, -0.73855921, ...,  0.01147543,
         0.89782053, -0.86813257]])

In [209]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(X_train_sj['city'])

X_train_sj['city']  = le.transform(X_train_sj['city'])
le.fit(X_test_sj['city'])
X_test_sj['city'] =le.transform(X_test_sj['city'])
le.fit(holdout_sj['city'])
holdout_sj['city'] =le.transform(holdout_sj['city'])


le.fit(X_train_iq['city'])

X_train_iq['city']  = le.transform(X_train_iq['city'])
le.fit(X_test_iq['city'])
X_test_iq['city'] =le.transform(X_test_iq['city'])
le.fit(holdout_iq['city'])
holdout_iq['city'] =le.transform(holdout_iq['city'])


In [210]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_sj,y_train_sj)
y_pred_sj = lr.predict(X_test_sj)
y_holdout_sj = lr.predict(holdout_sj)

lr.fit(X_train_iq,y_train_iq)
y_pred_iq = lr.predict(X_test_iq)
y_holdout_iq = lr.predict(holdout_iq)

In [211]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test_sj,y_pred_sj),mean_absolute_error(y_test_iq,y_pred_iq)

(28.930851063829788, 7.6057692307692308)

In [212]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=300)
random_forest.fit(X_train_sj, y_train_sj)
y_pred_sj = random_forest.predict(X_test_sj)
mean_absolute_error(y_test_sj,y_pred_sj)
y_holdout_sj = random_forest.predict(holdout_sj)

random_forest.fit(X_train_iq, y_train_iq)
y_pred_iq = random_forest.predict(X_test_iq)
mean_absolute_error(y_test_iq,y_pred_iq)
y_holdout_iq = random_forest.predict(holdout_iq)

df_test_sj.reset_index(drop=True, inplace=True)
df_test_iq.reset_index(drop=True, inplace=True)

y_sj = pd.concat([df_test_sj.iloc[:,0:3],pd.DataFrame(y_holdout_sj, columns=['total_cases'])], axis=1)
y_iq = pd.concat([df_test_iq.iloc[:,0:3],pd.DataFrame(y_holdout_iq, columns=['total_cases'])], axis=1)

final_y = y_sj.append(y_iq)

In [213]:
final_y


,city,year,weekofyear,total_cases
0,sj,2008,18,3
1,sj,2008,19,7
2,sj,2008,20,5
3,sj,2008,21,5
4,sj,2008,22,4
5,sj,2008,23,10
6,sj,2008,24,3
7,sj,2008,25,13
8,sj,2008,26,13
9,sj,2008,27,2


In [214]:
final_y.to_csv('submission.csv',index=False)
